In [82]:
import io
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm

In [83]:
path = 'dados/CAPTCHA-10k'
save_path = 'dados/processed'
labels_folder = 'labels10k'
if not os.path.exists(save_path):
    os.makedirs(save_path)
num_chars = 6

In [84]:
labels_dict = {}
labels_path = 'dados/CAPTCHA-10k/labels10k'

for fname in os.listdir(labels_path):
    if not fname.endswith('.txt'):
        continue

    base_name = os.path.splitext(fname)[0]
    full_path = os.path.join(labels_path, fname)

    with open(full_path, 'r') as f:
        content = f.read().strip()
        labels_dict[base_name] = content


In [85]:
def save_cleaned_image(path, folder, filename, save_path):
    img_path = os.path.join(path, folder, filename)
    img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img_gray is None:
        print(f"Erro: Não foi possível carregar a imagem em {img_path}")
        return

    # Remoção de ruído
    denoised_img = cv2.medianBlur(img_gray, 1)

    # Binarização (fundo preto, texto branco)
    threshold_value = 80
    _, binary_cleaned_img = cv2.threshold(denoised_img, threshold_value, 255, cv2.THRESH_BINARY_INV)

    # Erosão leve para tentar separar caracteres colados
    kernel = np.ones((2, 2), np.uint8)
    binary_cleaned_img = cv2.erode(binary_cleaned_img, kernel, iterations=2)

    
    kernel = np.ones((1, 1), np.uint8)
    binary_cleaned_img = cv2.dilate(binary_cleaned_img, kernel, iterations=1)

    # Criar pasta de saída
    output_folder_path = os.path.join(save_path, folder, 'cleanImage')
    os.makedirs(output_folder_path, exist_ok=True)

    # Salvar imagem final
    cleaned_filename = f"{os.path.splitext(filename)[0]}{os.path.splitext(filename)[1]}"
    full_output_path = os.path.join(output_folder_path, cleaned_filename)
    cv2.imwrite(full_output_path, binary_cleaned_img)

In [86]:
for folder in tqdm(os.listdir(path), desc="Processing folders"):
    if os.path.isdir(os.path.join(path, folder)) and folder != labels_folder:
        for filename in tqdm(os.listdir(os.path.join(path, folder)), desc=f"Cleaning {folder}", leave=False):
            if filename.endswith('.jpg'):
                save_cleaned_image(path, folder, filename, save_path)


Processing folders:   0%|          | 0/4 [00:00<?, ?it/s]

Processing folders: 100%|██████████| 4/4 [00:03<00:00,  1.24it/s]


In [87]:
def split_image(path, folder, filename, save_path,
                        margin=12, max_char_width=48, min_advance=8, debug=False):
    img_path = os.path.join(path, folder, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Erro: Não foi possível carregar a imagem em {img_path}")
        return

    height, width = img.shape
    step = width / 6
    cut_points = [int(round(i * step)) for i in range(7)]  

    os.makedirs(save_path, exist_ok=True)
    base_name, ext = os.path.splitext(filename)

    if base_name not in labels_dict or len(labels_dict[base_name]) != 6:
        print(f"Label ausente ou incompleto para: {base_name}")
        return

    label_seq = labels_dict[base_name]

    prev_end_x = 0  # Para controlar avanço mínimo entre cortes

    for i in range(6):
        # Garante que o próximo corte avance pelo menos 'min_advance' pixels
        base_start_x = max(prev_end_x + min_advance, cut_points[i])
        base_end_x = cut_points[i + 1]

        region = img[:, base_start_x:base_end_x]
        proj = np.sum(region, axis=0)

        region_width = base_end_x - base_start_x
        center_local = int(np.argmax(proj))

        # Limitar deslocamento do centro (máx 25% da largura da região)
        center_shift = center_local - region_width // 2
        max_shift = region_width // 4
        clamped_shift = int(np.clip(center_shift, -max_shift, max_shift))
        adjusted_center = base_start_x + region_width // 2 + clamped_shift

        # Define faixa de corte com margem
        start_x = int(max(0, adjusted_center - region_width // 2 - margin))
        end_x = int(min(width, adjusted_center + region_width // 2 + margin))

        # Impor limite máximo de largura
        if end_x - start_x > max_char_width:
            extra = (end_x - start_x - max_char_width) // 2
            start_x += extra
            end_x -= extra

        # Atualiza o fim da faixa atual para o controle do próximo corte
        prev_end_x = end_x

        char_img = img[:, start_x:end_x]
        char_label = label_seq[i]

        # Salvar
        save_path_full = os.path.join(save_path, char_label)
        os.makedirs(save_path_full, exist_ok=True)

        final_path = os.path.join(save_path_full, f"{base_name}_{i}{ext}")
        cv2.imwrite(final_path, char_img)

        # Debug visual
        if debug:
            img_debug = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            cv2.rectangle(img_debug, (start_x, 0), (end_x, height), (0, 255, 0), 2)
            cv2.imshow(f"Char {i}", img_debug)
            cv2.waitKey(0)

    if debug:
        cv2.destroyAllWindows()

In [88]:
conjuntos = ['treinamento', 'teste', 'validacao']
clean_base_path = '/home/bernardo/TP2-ICV/dados/processed'

for conjunto in conjuntos:
    # Caminho real: processed/treinamento/cleanImage
    full_clean_folder = os.path.join(clean_base_path, conjunto, 'cleanImage')

    for filename in tqdm(os.listdir(full_clean_folder), desc=f"Segmentando {conjunto}"):
        if filename.endswith('.jpg'):
            save_folder = os.path.join(clean_base_path, conjunto)

            split_image(
                path=full_clean_folder,   # caminho exato onde está a imagem limpa
                folder='',                # não precisa mais, o path já está completo
                filename=filename,
                save_path=save_folder,
                debug=False
            )


Segmentando treinamento:   5%|▌         | 436/8000 [00:00<00:08, 885.04it/s]

Label ausente ou incompleto para: 006343


Segmentando treinamento:  79%|███████▉  | 6340/8000 [00:07<00:01, 921.96it/s]

Label ausente ou incompleto para: 006742


Segmentando treinamento:  96%|█████████▌| 7650/8000 [00:09<00:00, 740.82it/s]

...

Segmentando teste: 100%|██████████| 1000/1000 [00:01<00:00, 846.17it/s]


Label ausente ou incompleto para: 009067


Segmentando validacao:  80%|███████▉  | 796/1000 [00:00<00:00, 917.76it/s] 

Label ausente ou incompleto para: 008714


Segmentando validacao: 100%|██████████| 1000/1000 [00:01<00:00, 904.68it/s]


In [ ]:
import os
import shutil
from tqdm import tqdm

# Caminho base dos dados
base_path = '/home/bernardo/TP2-ICV/dados'

# Pasta onde estão os arquivos organizados por classe
processed_path = os.path.join(base_path, 'processed')

# Pasta de saída (fora da pasta processed)
output_path = os.path.join(base_path, 'sequencial')
os.makedirs(output_path, exist_ok=True)

conjuntos = ['teste']

for conjunto in conjuntos:
    conjunto_path = os.path.join(processed_path, conjunto)
    for classe in os.listdir(conjunto_path):
        classe_path = os.path.join(conjunto_path, classe)
        if not os.path.isdir(classe_path):
            continue

        for filename in tqdm(os.listdir(classe_path), desc=f"Copiando {conjunto}/{classe}"):
            if filename.endswith('.jpg'):
                nome_base, ext = os.path.splitext(filename)

                # Monta o novo nome: conjunto_nomebase_label.jpg
                new_filename = f"{nome_base}_{classe}{ext}"

                src = os.path.join(classe_path, filename)
                dst = os.path.join(output_path, new_filename)

                shutil.copy(src, dst)


Copiando teste/U: 100%|██████████| 168/168 [00:00<00:00, 5845.48it/s]
